## Termcolor

Termcolor - это пакет Python для цветового форматирования ANSII для вывода текста в терминале. 

In [ ]:
!pip install termcolor

In [2]:
from termcolor import colored, cprint
print(colored('Привет мир!', 'red', attrs=['underline']))
print(colored('Привет', 'red', attrs=['underline']), colored('мир!', 'green'))
print('Привет, всем!')
cprint('Вывод с помощью cprint', 'green', 'on_blue')

Привет мир!
Привет мир!
Привет, всем!
Вывод с помощью cprint


## Numba 


https://numba.pydata.org/  
транслирует функцию в C  
Функцию нужно декарировать с помощью декоратора @git  
Особенно сильно ускореят, если в функции есть циклы  
например, мы декарируем функция и эта функция сначала скомпилируется в С и потом уже выполнится   
jit (just in time)  
Важно, чтобы добиться ускорения, на вход функции должен приходить numpy array  
Поэтому pandas series нужно преобразовывать перед передачей в функцию 

то есть, в функцию передаем `df.column_name.values`

In [1]:
!pip uninstall numba

^C


In [5]:
from numba import njit
import random
import numpy as np
import pandas as pd
import numba

@njit
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [2]:
@njit(parallel=True)
def logistic_regression(Y, X, w, iterations):
    for i in range(iterations):
        w -= np.dot(((1.0 /
              (1.0 + np.exp(-Y * np.dot(X, w)))
              - 1.0) * Y), X)
    return w

Смоделируем типичную ситуацию — нужно добавить новую колонку, применив какую-то функцию к уже существующей с помощью метода apply.

In [9]:
# создадим таблицу в 100 000 строк и 4 колонки, заполненную случайными числами от 0 до 100
df = pd.DataFrame(np.random.randint(0,100,size=(100000, 4)),columns=['a', 'b', 'c', 'd'])

# функция для создания новой колонки
def multiply(x):
    return x * 5
    
# оптимизированная с помощью numba версия
@numba.vectorize
def multiply_numba(x):
    return x * 5

# наша функция
%timeit df['new_col'] = df['a'].apply(multiply)

# встроенная имплементация Pandas
%timeit df['new_col'] = df['a'] * 5

# наша функция с numba
# мы отдаем весь вектор значений, чтобы numba сам провел оптимизацию цикла
%timeit df['new_col'] = multiply_numba(df['a'].to_numpy())

105 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.2 ms ± 395 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.68 ms ± 994 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


В принципе пандас неплохо справился, но   
если будет посложнее

In [10]:
# возводим значения строки в квадрат и берем их среднее 
def square_mean(row):
    row = np.power(row, 2)
    return np.mean(row)

%timeit df['new_col'] = df.apply(square_mean, axis=1)


@numba.njit
def square_mean_numba(arr):
    res = np.empty(arr.shape[0])
    arr = np.power(arr, 2)
    for i in range(arr.shape[0]):
        res[i] = np.mean(arr[i])
    return res

# numba не умеет работать с примитивами pandas (Dataframe, Series и тд.)
# поэтому мы даем ей двумерный массив numpy
%timeit df['new_col'] = square_mean_numba(df.to_numpy())

47.5 s ± 7.18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
12.4 ms ± 799 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Modin

https://github.com/modin-project/modin

Modin занимается распараллеливанием вычислений, но вместе с этим позволяет использовать дисковое пространство, когда оперативки не хватает  
Опция использования дискового пространства включена по дефолту

modin разбивает датафрейм на куски, что позволяет ему записывать данные на диск,    
но это замедляет работу, даже если памяти хватает, так как теперь нужно не только найти столбец, но ещё кусок этого столбца при индексации  
Поэтому используем только, когда не хватает памяти 

При размере файла меньше 1Гб не стоит использовать, эффекта может не быть

In [ ]:
pip install "modin[all]" # (Recommended) Install Modin with Ray and Dask engines.

Импортируем и работаем как с обычным `pandas`

In [ ]:
import modin.pandas as pd


## Folium

Folium — это мощная библиотека визуализации данных в Python, которая была создана в первую очередь для того,  
чтобы помочь людям визуализировать гео-пространственные данные.  
https://python-visualization.github.io/folium/latest/getting_started.html

In [ ]:
!pip install folium

In [2]:
import folium
import pandas as pd

In [5]:
df = pd.DataFrame(data={
    'lat': [55.75417, 55.4537, 55.7481389],
    'lon': [37.62, 37.3701, 37.5371719],
    'type': ['Красная площадь', 'Большой театр', 'Москва Сиити']
})
df

,lat,lon,type
0,55.754170,37.620000,Красная площадь
1,55.453700,37.370100,Большой театр
2,55.748139,37.537172,Москва Сиити


In [ ]:
# mp = folium.Map(location=(df['lat'].mean(), df['lon'].mean()), zoom_start=13)  # , 
mp = folium.Map(location=(55.75417, 37.62), zoom_start=13)  # , 
mp

In [ ]:
# popup - это что будет показываться, когда нажать на маркер на карте
# icon - это настройки маркеров
def add_marker(row, mp):
    folium.Marker([row['lat'], row['lon']], 
                  popup=row['type'], 
                  icon=folium.Icon(color="green" if row['type'] == 'Москва Сиити' else 'blue')).add_to(mp)
    
df.apply(add_marker, args=[mp], axis=1);
mp

## urllib

In [1]:
from urllib.parse import urljoin

Если у нас есть 2 урл и мы хотим их соеденить

In [5]:
url1 = 'https://my_cite.com/cat1/cat2/cat3/index.html'
url2 = '../../../cat4/cat5pic.jpg'
urljoin(url1, url2)

'https://my_cite.com/cat4/cat5pic.jpg'

## tqdm

https://tqdm.github.io/docs/tqdm/

In [1]:
from tqdm.auto import tqdm

Если мы хотим увидеть бар с динамикой выполнения кода, то  
нужно `tqdm` поместить в for 

- iterable: iterable, optional  
Iterable to decorate with a progressbar. Leave blank to manually manage the updates.  
- desc: str, optional  
Prefix for the progressbar.  

In [4]:
for i in tqdm(iterable=range(1, 100000), desc='my_prefix'):
    i*i

my_prefix:   0%|          | 0/99999 [00:00<?, ?it/s]